# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [20]:
## Plotly is not included in your dojo-env
!pip install plotly

In [21]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [22]:
## Load in csv.gz
df = pd.read_csv("Data/Seattle-Pizza.csv.gz")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             840 non-null    object 
 1   alias          840 non-null    object 
 2   name           840 non-null    object 
 3   image_url      828 non-null    object 
 4   is_closed      840 non-null    bool   
 5   url            840 non-null    object 
 6   review_count   840 non-null    int64  
 7   categories     840 non-null    object 
 8   rating         840 non-null    float64
 9   coordinates    840 non-null    object 
 10  transactions   840 non-null    object 
 11  location       840 non-null    object 
 12  phone          807 non-null    float64
 13  display_phone  807 non-null    object 
 14  distance       840 non-null    float64
 15  price          750 non-null    object 
dtypes: bool(1), float64(3), int64(1), object(11)
memory usage: 99.4+ KB


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,24,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176,NaN
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,344,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,77,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","['pickup', 'delivery']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,966,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","['pickup', 'delivery']","{'address1': '1028 Madison St', 'address2': No...",1.206538e+10,(206) 538-0040,2271.307105,$$
4,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,267,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 47.62323, 'longitude': -122.29593}",['delivery'],"{'address1': '2743 E Madison St', 'address2': ...",1.206710e+10,(206) 709-8883,1130.900743,$$


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [23]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.61849, 'longitude': -122.31664}"
1,"{'latitude': 47.5976491915013, 'longitude': -1..."
2,"{'latitude': 47.624577, 'longitude': -122.325577}"
3,"{'latitude': 47.60937, 'longitude': -122.32546}"
4,"{'latitude': 47.62323, 'longitude': -122.29593}"
...,...
835,"{'latitude': 47.667511119207, 'longitude': -12..."
836,"{'latitude': 47.6157033283809, 'longitude': -1..."
837,"{'latitude': 47.55172, 'longitude': -122.36327}"
838,"{'latitude': 47.676215, 'longitude': -122.205084}"


- Why didn't that work???

In [24]:
## slice out a single test coordinate
test_coord = df.loc[0,'coordinates']
test_coord

"{'latitude': 47.61849, 'longitude': -122.31664}"

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

In [25]:

type(test_coord)

str

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [26]:
## Use json.loads on the test coordinate
#json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [27]:
## replace single ' with " 
test_coord = test_coord.replace("'",'"')

In [28]:
test_coord

'{"latitude": 47.61849, "longitude": -122.31664}'

In [29]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 47.61849, 'longitude': -122.31664}

### Now, how can we apply this same process to the entire column??

In [30]:
## replace ' with " (entire column)
#df['coordinates'].str.replace("'",'"')


## apply json.loads


In [31]:
# method 2
#df['coordinates'] = df['coordinates'].apply(lambda cell : cell.replace("'",'"'))

In [32]:
#df['coordinates'] = df['coordinates'].apply(json.loads)

In [33]:
#df['coordinates'].apply(pd.Series)

In [34]:
lat_long = df['coordinates'].apply(lambda cell : pd.Series((json.loads(cell.replace("'",'"')))))

In [35]:
lat_long

,latitude,longitude
0,47.618490,-122.316640
1,47.597649,-122.313306
2,47.624577,-122.325577
3,47.609370,-122.325460
4,47.623230,-122.295930
...,...,...
835,47.667511,-122.375935
836,47.615703,-122.200735
837,47.551720,-122.363270
838,47.676215,-122.205084


In [ ]:
## slice out a single test coordinate


### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [ ]:
## use .apply pd.Series to convert a dict to columns
#df['coordinates'].apply(pd.Series)

In [36]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df.drop(columns = ['coordinates']),lat_long],axis =1)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,24,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,344,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,77,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,966,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']","{'address1': '1028 Madison St', 'address2': No...",1.206538e+10,(206) 538-0040,2271.307105,$$,47.609370,-122.325460
4,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,267,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,['delivery'],"{'address1': '2743 E Madison St', 'address2': ...",1.206710e+10,(206) 709-8883,1130.900743,$$,47.623230,-122.295930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,7tQY7TW-oYhAztdpUVgfZQ,subway-seattle-127,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/KFiJVP...,False,https://www.yelp.com/biz/subway-seattle-127?ad...,39,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",3.0,"['pickup', 'delivery']","{'address1': '5300 15th Ave NW', 'address2': '...",1.206790e+10,(206) 789-5434,8529.998816,$,47.667511,-122.375935
836,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,63,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{'address1': '600 Bellevue Way NE', 'address2'...",1.425638e+10,(425) 638-1000,7123.208902,$$,47.615703,-122.200735
837,G-p5SgQnBM5e8XxMMO1qOg,pho-aroma-seattle-2,Pho Aroma,https://s3-media1.fl.yelpcdn.com/bphoto/BCxDac...,False,https://www.yelp.com/biz/pho-aroma-seattle-2?a...,384,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,['delivery'],"{'address1': '5605 Delridge Way SW', 'address2...",1.206932e+10,(206) 932-4343,8499.808775,$$,47.551720,-122.363270
838,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,184,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"['delivery', 'pickup']","{'address1': '147 Park Ln', 'address2': '', 'a...",1.425577e+10,(425) 576-5407,9765.353049,$$,47.676215,-122.205084


## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [37]:
## Load in mapbox api credentials from .secret
with open('/Users/purvikansara/.secret/mapbox.json') as f:
    creds = json.load(f)
creds.keys()

dict_keys(['client-id', 'api-key'])

- Use the plotly express `set_maptbox_acccess_token` function

In [38]:
## set mapbox token
px.set_mapbox_access_token(creds['api-key'])


In [42]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df, lat = 'latitude',lon = 'longitude',mapbox_style='open-street-map',color = 'rating')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [44]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df, lat = 'latitude',lon = 'longitude',mapbox_style='open-street-map',
                  hover_name= 'name',hover_data=['price','location','transactions'])

### Fixing the Location Column

In [45]:
## slice out a test address
test_addr = df.loc[0,'location']
test_addr

"{'address1': '1830 12th Ave', 'address2': '', 'address3': None, 'city': 'Seattle', 'zip_code': '98122', 'country': 'US', 'state': 'WA', 'display_address': ['1830 12th Ave', 'Seattle, WA 98122']}"

> Also a string-dictionary...

In [47]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,24,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,344,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,77,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,966,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']","{""address1"": ""1028 Madison St"", ""address2"": No...",1.206538e+10,(206) 538-0040,2271.307105,$$,47.609370,-122.325460
4,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,267,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,['delivery'],"{""address1"": ""2743 E Madison St"", ""address2"": ...",1.206710e+10,(206) 709-8883,1130.900743,$$,47.623230,-122.295930


In [49]:
## apply json.loads
df['location'].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 59 (char 58)

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [50]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "1830 12th Ave", "address2": "", "address3": None, "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["1830 12th Ave", "Seattle, WA 98122"]}'

In [51]:
result = test_addr.split('address2')
result[0]

'{"address1": "1830 12th Ave", "'

In [52]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return "ERROR"

In [53]:
## test applying our function
df['location'].apply(fix_address)  

0                                                  ERROR
1      {'address1': '525 Rainier Ave S', 'address2': ...
2                                                  ERROR
3                                                  ERROR
4      {'address1': '2743 E Madison St', 'address2': ...
                             ...                        
835    {'address1': '5300 15th Ave NW', 'address2': '...
836    {'address1': '600 Bellevue Way NE', 'address2'...
837    {'address1': '5605 Delridge Way SW', 'address2...
838    {'address1': '147 Park Ln', 'address2': '', 'a...
839                                                ERROR
Name: location, Length: 840, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [55]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,24,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640,ERROR
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,344,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ..."
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,77,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577,ERROR
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,966,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']","{""address1"": ""1028 Madison St"", ""address2"": No...",1.206538e+10,(206) 538-0040,2271.307105,$$,47.609370,-122.325460,ERROR
4,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,267,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,['delivery'],"{""address1"": ""2743 E Madison St"", ""address2"": ...",1.206710e+10,(206) 709-8883,1130.900743,$$,47.623230,-122.295930,"{'address1': '2743 E Madison St', 'address2': ..."


In [56]:
## filter for businesses with display_location == "ERROR"
df.loc[df['display_location']=='ERROR']

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,24,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640,ERROR
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,77,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577,ERROR
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,966,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']","{""address1"": ""1028 Madison St"", ""address2"": No...",1.206538e+10,(206) 538-0040,2271.307105,$$,47.609370,-122.325460,ERROR
6,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,99,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,['delivery'],"{""address1"": ""4526 42nd Ave SW"", ""address2"": ""...",1.206421e+10,(206) 420-8880,8784.075722,NaN,47.562070,-122.385090,ERROR
8,-FOAQv22SXtSBs7nptI3UA,serious-pie-downtown-seattle-2,Serious Pie Downtown,https://s3-media2.fl.yelpcdn.com/bphoto/dy0pJ5...,False,https://www.yelp.com/biz/serious-pie-downtown-...,4355,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']","{""address1"": ""2001 4th Ave"", ""address2"": None,...",1.206839e+10,(206) 838-7388,3367.748997,$$,47.612858,-122.340611,ERROR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,Nihv2RfuTZ4TI8BQl3anmA,qfc-kirkland-4,QFC,https://s3-media2.fl.yelpcdn.com/bphoto/NYz4rH...,False,https://www.yelp.com/biz/qfc-kirkland-4?adjust...,29,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,[],"{""address1"": ""457 Central Way"", ""address2"": """"...",1.425827e+10,(425) 827-2205,10212.263314,$$,47.678048,-122.199308,ERROR
827,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/-o6eAW...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,35,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],"{""address1"": ""400 Bellevue Way NE"", ""address2""...",1.425372e+10,(425) 372-0250,7071.129218,$,47.614080,-122.201363,ERROR
830,AsgtzyqpdjuPyjaEUVcUXA,jewel-of-india-seattle-3,Jewel Of India,https://s3-media2.fl.yelpcdn.com/bphoto/qJGMnl...,False,https://www.yelp.com/biz/jewel-of-india-seattl...,396,"[{'alias': 'indpak', 'title': 'Indian'}]",3.5,"['pickup', 'delivery']","{""address1"": ""4735 University Way NE"", ""addres...",1.206524e+10,(206) 523-5275,5825.461180,$$,47.664103,-122.313426,ERROR
831,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,253,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['delivery', 'pickup']","{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,$$,47.612521,-122.201333,ERROR


In [58]:
## slice out a new test address and inspect
test_addr = df.loc[831, 'location']
test_addr

'{"address1": "222 Bellevue Way NE", "address2": None, "address3": "", "city": "Bellevue", "zip_code": "98004", "country": "US", "state": "WA", "display_address": ["222 Bellevue Way NE", "Bellevue, WA 98004"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [59]:
## remove any rows where display_location == 'ERROR'
df = df.loc[df['display_location']!='ERROR']
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,344,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ..."
4,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,267,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,['delivery'],"{""address1"": ""2743 E Madison St"", ""address2"": ...",1.206710e+10,(206) 709-8883,1130.900743,$$,47.623230,-122.295930,"{'address1': '2743 E Madison St', 'address2': ..."
5,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,178,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ..."
7,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/pcUxq_...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,917,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ..."
9,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,319,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,TJLPq5XM2gU3whWksAR5iw,kfc-seattle-8,KFC,https://s3-media3.fl.yelpcdn.com/bphoto/i_582D...,False,https://www.yelp.com/biz/kfc-seattle-8?adjust_...,52,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"['pickup', 'delivery']","{""address1"": ""1140 NW Market St"", ""address2"": ...",1.206789e+10,(206) 789-0282,8478.426534,$,47.668896,-122.372807,"{'address1': '1140 NW Market St', 'address2': ..."
835,7tQY7TW-oYhAztdpUVgfZQ,subway-seattle-127,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/KFiJVP...,False,https://www.yelp.com/biz/subway-seattle-127?ad...,39,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",3.0,"['pickup', 'delivery']","{""address1"": ""5300 15th Ave NW"", ""address2"": ""...",1.206790e+10,(206) 789-5434,8529.998816,$,47.667511,-122.375935,"{'address1': '5300 15th Ave NW', 'address2': '..."
836,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,63,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{""address1"": ""600 Bellevue Way NE"", ""address2""...",1.425638e+10,(425) 638-1000,7123.208902,$$,47.615703,-122.200735,"{'address1': '600 Bellevue Way NE', 'address2'..."
837,G-p5SgQnBM5e8XxMMO1qOg,pho-aroma-seattle-2,Pho Aroma,https://s3-media1.fl.yelpcdn.com/bphoto/BCxDac...,False,https://www.yelp.com/biz/pho-aroma-seattle-2?a...,384,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,['delivery'],"{""address1"": ""5605 Delridge Way SW"", ""address2...",1.206932e+10,(206) 932-4343,8499.808775,$$,47.551720,-122.363270,"{'address1': '5605 Delridge Way SW', 'address2..."


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [60]:

df.loc[838,'display_location']

{'address1': '147 Park Ln',
 'address2': '',
 'address3': '',
 'city': 'Kirkland',
 'zip_code': '98033',
 'country': 'US',
 'state': 'WA',
 'display_address': ['147 Park Ln', 'Kirkland, WA 98033']}

In [62]:
## use apply and lambda to slice correct key
df['display_address']= df['display_location'].apply(lambda x : x['display_address'])

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_29028/2289827437.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [63]:
## slice out a test_address
test = df.loc[4,'display_address']
test

['2743 E Madison St', 'Seattle, WA 98112']

In [64]:
## test using .join with a "\n"
print("\n".join(test))

2743 E Madison St
Seattle, WA 98112


In [65]:
## apply the join to every row with a lambda
df['Address'] = df['display_address'].apply(lambda x : "\n".join(x))
df

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_29028/2929995131.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location,display_address,Address
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,344,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]","525 Rainier Ave S\nSeattle, WA 98144"
4,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,267,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,['delivery'],"{""address1"": ""2743 E Madison St"", ""address2"": ...",1.206710e+10,(206) 709-8883,1130.900743,$$,47.623230,-122.295930,"{'address1': '2743 E Madison St', 'address2': ...","[2743 E Madison St, Seattle, WA 98112]","2743 E Madison St\nSeattle, WA 98112"
5,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,178,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]","4235 E Madison St\nSeattle, WA 98112"
7,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/pcUxq_...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,917,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]","700 E Pine St\nSeattle, WA 98122"
9,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,319,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]","1546 15th Ave\nSeattle, WA 98122"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,TJLPq5XM2gU3whWksAR5iw,kfc-seattle-8,KFC,https://s3-media3.fl.yelpcdn.com/bphoto/i_582D...,False,https://www.yelp.com/biz/kfc-seattle-8?adjust_...,52,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"['pickup', 'delivery']","{""address1"": ""1140 NW Market St"", ""address2"": ...",1.206789e+10,(206) 789-0282,8478.426534,$,47.668896,-122.372807,"{'address1': '1140 NW Market St', 'address2': ...","[1140 NW Market St, Seattle, WA 98107]","1140 NW Market St\nSeattle, WA 98107"
835,7tQY7TW-oYhAztdpUVgfZQ,subway-seattle-127,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/KFiJVP...,False,https://www.yelp.com/biz/subway-seattle-127?ad...,39,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",3.0,"['pickup', 'delivery']","{""address1"": ""5300 15th Ave NW"", ""address2"": ""...",1.206790e+10,(206) 789-5434,8529.998816,$,47.667511,-122.375935,"{'address1': '5300 15th Ave NW', 'address2': '...","[5300 15th Ave NW, Seattle, WA 98107]","5300 15th Ave NW\nSeattle, WA 98107"
836,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,63,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{""address1"": ""600 Bellevue Way

### Final Map

In [67]:
## make ourn final map and save as varaible
fig = px.scatter_mapbox(df, lat = 'latitude',lon = 'longitude',mapbox_style='open-street-map',
                  hover_name= 'name',hover_data=['price','Address','transactions'],color = 'rating')

In [68]:
fig

#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map

In [69]:
## use fig.write_html to save map
fig.write_html('final_map.html')